<a href="https://colab.research.google.com/github/attila5287/scrap_and_dataviz/blob/main/states_ue.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install matplotlib pandas requests folium geopandas

     |████████████████████████████████| 1.0 MB 7.6 MB/s 
     |████████████████████████████████| 16.7 MB 18.6 MB/s 
     |████████████████████████████████| 6.3 MB 20.4 MB/s 


In [2]:
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import pandas as pd
import json
import requests
import folium
from folium.plugins import TimeSliderChoropleth
import geopandas as gpd

In [3]:
urljson = \
'https://raw.githubusercontent.com/shawnbot/topogram/master/data/us-states.geojson'

In [4]:
gdf = gpd.read_file( requests.get(urljson).text )
gdf = gdf[[ 'name', 'geometry' ]] \
        .sort_values('name')
len(gdf)
gdf.head()

,name,geometry
28,Alabama,"POLYGON ((-85.62539 34.98590, -85.36556 33.744..."
47,Alaska,"MULTIPOLYGON (((-153.00631 57.11584, -154.0050..."
7,Arizona,"POLYGON ((-109.04523 36.99992, -109.04367 31.3..."
14,Arkansas,"POLYGON ((-89.66292 36.02198, -89.67351 35.939..."
8,California,"POLYGON ((-119.99897 41.99261, -120.00003 38.9..."


In [5]:
gdf['state'] = [
       name.replace(' ', '') for name in gdf['name']
]


In [6]:
gdf

,name,geometry,state
28,Alabama,"POLYGON ((-85.62539 34.98590, -85.36556 33.744...",Alabama
47,Alaska,"MULTIPOLYGON (((-153.00631 57.11584, -154.0050...",Alaska
7,Arizona,"POLYGON ((-109.04523 36.99992, -109.04367 31.3...",Arizona
14,Arkansas,"POLYGON ((-89.66292 36.02198, -89.67351 35.939...",Arkansas
8,California,"POLYGON ((-119.99897 41.99261, -120.00003 38.9...",California
9,Colorado,"POLYGON ((-102.04709 40.00081, -102.03858 38.4...",Colorado
23,Connecticut,"POLYGON ((-72.87617 41.22055, -73.64789 40.953...",Connecticut
41,Delaware,"POLYGON ((-75.40557 39.79550, -75.55427 39.691...",Delaware
49,District of Columbia,"POLYGON ((-77.04123 38.78955, -77.04123 38.789...",DistrictofColumbia
29,Florida,"POLYGON ((-81.49047 30.72983, -81.31371 30.035...",Florida


In [7]:
url = 'https://www.ncsl.org/research/labor-and-employment/state-unemployment-update.aspx'

In [8]:
raw_df = pd.read_html(url)[0]

In [9]:
# raw_df

In [10]:
for month in raw_df.transpose().columns:
  pass
  # print( raw_df.set_index('State') )

In [11]:
l = []
for i,state in enumerate(raw_df['State']):
    pass
    d={}
    for j, month in enumerate( raw_df.columns[1:] ):
        pass
        d= {
            'State' : state,
            'Month' : month.replace('.', ''),
            # 'Date'  : dt_index[ j ],  
            'Rate' :  list(
                raw_df.loc[ raw_df['State'] == state ][month])[0],
        }
        l.append(d)
        
df = pd.DataFrame(l)    

In [99]:
df.loc[df['State'] == 'D.C.', 'State'] = 'District of Columbia'

In [102]:
df['name'] = [
              state.replace(' ', '') for state in df['State']
]

In [103]:
df['name'].unique()

array(['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California',
       'Colorado', 'Connecticut', 'Delaware', 'DistrictofColumbia',
       'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana',
       'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland',
       'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi',
       'Missouri', 'Montana', 'Nebraska', 'Nevada', 'NewHampshire',
       'NewJersey', 'NewMexico', 'NewYork', 'NorthCarolina',
       'NorthDakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania',
       'RhodeIsland', 'SouthCarolina', 'SouthDakota', 'Tennessee',
       'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington',
       'WestVirginia', 'Wisconsin', 'Wyoming', 'PuertoRico'], dtype=object)

In [104]:
nan_states = list(df.loc[
        df['Rate'].isnull()
 ]['State'])
nan_states

[]

In [105]:
avg_list = [
            round(df.loc[ df['State'] == state ].mean()['Rate'], 2)\
            for state in nan_states
            
            ] 
avg_list

[]

In [106]:
# df.loc[ df['Rate'].isnull()]['Rate'] = avg_list
df.loc[ df['Rate'].isnull()]['Rate'] 
# values = { "A": 0, "B": 1, "C": 2, "D": 3 }
df.fillna(value=3.0, inplace=True)
df.loc[ df['Rate'].isnull()]['Rate'] 

Series([], Name: Rate, dtype: float64)

In [107]:
maxrate = df.max()['Rate']
minrate = df.min()['Rate']

In [108]:
df['scaled'] = [
  (rate-minrate)/(maxrate-minrate) for rate in df['Rate']
]

In [109]:
from branca.colormap import linear
cmap = linear.BuPu_09.scale( 0.00, maxrate)
cmap

In [110]:
df['opacity'] = 0.99

In [111]:
df['color'] = df['Rate']
df[ 'color' ] = df[ 'color' ].apply(cmap)
# df['color'].unique()

In [112]:
datetime_index = pd.date_range("2021-1-1", periods=12, freq="M")
dt_index_epochs = datetime_index.astype(int) // 10 ** 9
dt_index = dt_index_epochs.astype("U10")

dt_index


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: casting datetime64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.
  


Index(['1612051200', '1614470400', '1617148800', '1619740800', '1622419200',
       '1625011200', '1627689600', '1630368000', '1632960000', '1635638400',
       '1638230400', '1640908800'],
      dtype='object')

In [116]:
d = {}

for i,state in enumerate(gdf['state']):
  pass
  v = df.loc[ df['name'] == state ][[ 'color', 'opacity']]\
  .set_index(dt_index)\
  .to_dict(orient="index")
  d[state] = v

# d['NorthDakota']
# d['Colorado']

In [114]:
import folium
from folium.plugins import TimeSliderChoropleth

m = folium.Map([18, -90], tiles="Stamen Toner", zoom_start=3)

g = TimeSliderChoropleth(
    gdf.set_index('state').to_json(),
    styledict=d,
).add_to(m)

m

In [115]:
# folium.map.CustomPane("labels").add_to(m)

# # Final layer associated to custom pane via the appropriate kwarg
# folium.TileLayer("stamentonerlabels", pane="labels").add_to(m)
